In [1]:
'''This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classication of newsgroup messages into 20 different categories).
GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)
20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html
'''

'This script loads pre-trained word embeddings (GloVe embeddings)\ninto a frozen Keras Embedding layer, and uses it to\ntrain a text classification model on the 20 Newsgroup dataset\n(classication of newsgroup messages into 20 different categories).\nGloVe embedding data can be found at:\nhttp://nlp.stanford.edu/data/glove.6B.zip\n(source page: http://nlp.stanford.edu/projects/glove/)\n20 Newsgroup data can be found at:\nhttp://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html\n'

In [2]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras_tqdm import TQDMNotebookCallback
import pandas as pd

Using TensorFlow backend.


In [3]:

#Set Which GPU to use. 
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [16]:
#BASE_DIR = '/media/sohn/Storage/data_glove/'
DATA_PATH = '/data/sohn/wabd_full1.csv'
BASE_DIR = '/data/sohn/data_glove/'
GLOVE_DIR = BASE_DIR + 'glove.6b/'
TEXT_DATA_DIR = BASE_DIR + '20_newsgroup/'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [17]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [68]:
# second, prepare text samples and their labels
print('Processing text dataset')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Processing text dataset
Found 19997 texts.


In [18]:
#np.loadtxt(DATA_PATH, delimiter=',')
#pd.read_csv(DATA_PATH)
df = pd.read_excel('/data/sohn/wabd_full.xlsx')
texts = []
labels = []
texts=list(df['indication'])
labels=list(df['protocol1'])

In [19]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1125 unique tokens.


In [20]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
#labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (250, 1000)
Shape of label tensor: (250, 9)


In [35]:
texts[1:10]

['69 YO FEMALE FOUND TO HAVE PANCREATIC HEAD MASS ON CT, C/F PSEUDOCYST VS MALIGNANCY.  Reason: EVALUATE PANCREATIC HEAD MASS',
 'SBO, CT NONCONT SHOWED POSSIBLE MASS AT HEAD OF PANCREAS AND CYST LESION AT TAIL. HAS CKD.  Reason: MRCP PLEASE. EVALUATE POSSIBLEPANCREATIC MASSES',
 '54 Y/O F WITH PANCREATIC MASS ON CT  Reason: FURTHER IMAGING OF PATIENT PANCREATIC MASS',
 'MRCP - 72 YO M WITH NEW PANCREATIC HEAD MASS AND ?PANCREATITIS. PLEASE PERFORM MRCP TO EVALUATE PANCREATIC HEAD MASS  Reason: MRCP - PANCREATIC MASS',
 'EVALUATE FOR GALLSTONE VS MASS PANCREATITIS. RECOMMENDED PER GI CONSULT.  Reason: MRCP',
 '63M W/ DM2 ADMITTED WITH FOOT ULCER, FOUND TO HAVE WASTING, FTT, AND COGNITIVE IMPAIRMENT, WITH CHRONIC ALK PHOS ELEVATION AND "COARSE" LIVER ON U/S WITH DILATED BILE DUCT.  Reason: BILIARY DILATATION AND ELEVATED ALK PHOS, CONCERN FOR MASS/S; MRCP, LIVER',
 '57 Y/O F WITH PAINLESS JAUNDICE, CONCERN FOR PANCREATIC HEAD MASS ON U/S.  Reason: MRCP',
 '58 YO F W/MET CA (PRIMARY UNKN

In [56]:
np.asarray(labels)

array(['MRI/MRCP abdomen eovist', 'MRI abdomen gadavist',
       'MRI/MRCP abdomen eovist', 'MRI abdomen eovist',
       'MRI abdomen gadavist', 'MRI/MRCP abdomen eovist',
       'MRI abdomen gadavist', 'MRI/MRCP abdomen eovist',
       'MRI/MRCP abdomen gadavist', 'MRI/MRCP abdomen eovist',
       'MRI/MRCP abdomen gadavist', 'MRI abdomen eovist',
       'MRI Pelvis with and without', 'MRI appendicitis',
       'MRI abdomen gadavist', 'MRI Pelvis with and without',
       'MRI/MRCP abdomen eovist', 'MRI/MRCP abdomen eovist',
       'MRI/MRCP abdomen with gadavist', 'MRI Pelvis with and without',
       'MRI abdomen & pelvis gadavist', 'MRI abdomen eovist',
       'MRI abdomen eovist', 'MRA abdomen with gadavist',
       'MRI/MRCP abdomen gadavist', 'MRI/MRCP abdomen eovist',
       'MRA abdomen with gadavist', 'MRI/MRCP abdomen eovist',
       'MRI Pelvis with and without', 'MRI appendicitis',
       'MRI abdomen & pelvis gadavist', 'MRI/MRCP abdomen gadavist',
       'MRI appendiciti

In [21]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [22]:
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [23]:
print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


In [24]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [25]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(9, activation='softmax')(x)
#must change the last number to number of categories


Training model.


In [26]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 100)         112500    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 35, 128)           82048     
__________

In [28]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val),
          verbose=0,
          callbacks=[TQDMNotebookCallback(leave_inner=True)])

NameError: name 'x_test' is not defined